<a href="https://colab.research.google.com/github/moubarak1ezzyani/Pr-diction-du-Churn-Client----Git-Folder/blob/main/ClassificationImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chargement et préparation des données
## Charger Fashion MNIST via Tensorflow.keras.datasets


In [3]:
import tensorflow as tf

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Inspecter dimensions, classes

In [4]:
print(f"Dimension des images d'entrainement :{train_images.shape} ")
print(f"Dimension des etiquettes d'entrainement :{train_labels.shape} ")
print(f"Dimension des images de test :{test_images.shape} ")
print(f"Dimension des etiquettes de test :{test_labels.shape} ")
# train_image[0]

Dimension des images d'entrainement :(60000, 28, 28) 
Dimension des etiquettes d'entrainement :(60000,) 
Dimension des images de test :(10000, 28, 28) 
Dimension des etiquettes de test :(10000,) 


## Normaliser les pixels (diviser par 255 → valeurs dans [0,1]).

In [5]:
print(f"Max avant Standarisation : {train_images.max()}")

train_images = train_images / 255
test_images = test_images / 255

print(f"Max apres Standarisation : {train_images.max()}")

Max avant Standarisation : 255
Max apres Standarisation : 1.0


## Choix du format des labels : utiliser labels entiers (0–9) & loss sparse_categorical__crossentropy

In [6]:
import numpy as np
import tensorflow as tf

#choix format
classes_uniques = np.unique(train_labels)
print(f"labels of training : {len(classes_uniques)} classes uniques")
print(f"Ces classes sont : {classes_uniques}")


labels of training : 10 classes uniques
Ces classes sont : [0 1 2 3 4 5 6 7 8 9]


# Construction du modèle
## Architecture demandée
### Flatten(input_shape) → Dense(neurals number , activation function) → Dense(neurals number , activation function)

In [7]:
import numpy as np
import tensorflow as tf

#loss
#---- Creer un modele
model = tf.keras.Sequential([
    # 1. La couche d'entrée : Aplatir => Unidimensionnel
    tf.keras.layers.Flatten(input_shape=(28,28)),

    # 2. La couche cachée : Apprendre
    # 128 neurones. 'relu' est une fonction d'activation très commune.
    tf.keras.layers.Dense(128, activation='relu'),

    # 3. La couche de sortie : Prédire
    # 10 neurones, car il y est 10 classes (0-9).
    # 'softmax' transforme les scores en probabilités (ex: 80% de chance que ce soit un '9')
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()



/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

## Compiler :

### optimizer='optimizer_name', loss='loss function', metrics=['accuracy'].

In [8]:
#--- Compilation Models
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
print("===== modele cree ET compile avec succes !===== ")

===== modele cree ET compile avec succes !===== 


### Bonus - Ajouter des callbacks (EarlyStopping sur val_loss)


In [9]:
from tensorflow.keras.callbacks import EarlyStopping

# ------------- Definition de CallBack EarlyStpping
mon_assistant_callback = EarlyStopping(
    monitor='val_loss',     # La metric à surveiller
    patience = 3,           # attendre 3 epohcs (sans amelioration) avant de s'arreter
    restore_best_weights = True,
    verbose = 1             # Pour afficher le message lors d'arret
)



# Entraînement
## Entrainer 5 à 10 époques (montrer history).

In [3]:
import tensorflow as tf

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# -------------- Entrainement Modele avaec CallBack
print("========== Debut Entrainement =============")

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

#il doit parcourir la liste callback
history = model.fit(
    train_images,
    train_labels,
    epochs = 50,      # On met un nombre eleve, surement il va arreter avant
    validation_data = (test_images, test_labels),   # Indispensable , pour que val_loss existe
    callbacks = [mon_assistant_callback]        # c'est ici qu'on le passe
)
print("Entrainement terminé ")

#Evaluation
from sklearn.model_selection import train_test_split

results = model.evaluate (X_test, y_test)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
========== Debut Entrainement =============


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

NameError: name 'mon_assistant_callback' is not defined

## Évaluation
### Calculer la précision finale sur le jeu de test (model.evaluate).

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split # <--- Outil pour couper les données
from sklearn.datasets import load_iris

results = model.evaluate (X_test, y_test)

NameError: name 'model' is not defined